In [16]:
import pandas as pd
import datetime,csv
from tkinter import filedialog

# sklearn modules for data preprocessing-------------------------------------
from sklearn.preprocessing import StandardScaler 
import joblib

In [19]:
class inference():
    def __init__(self,database=''):
        database=pd.read_csv('dataset.csv')
        database = database.iloc[:-3652]
        self.database=database

    def getdata(self):
        print('How do you want to input:')
        ### Manual input---------------------------------------------
        # value = input("1: Choose a list of id from file (*.log, *.txt, *.csv); anything else for inference all in Database:\n")
        value=2

        id_list = []
        if value == 1:
            filename = filedialog.askopenfile(title='Please choose id list file')
            filename= filename.name
            # filename='1idlist.txt'
            with open(filename,'r',newline='') as f:
                    reader = csv.reader(f)
                    for row in reader:
                        id_list+=row  
        
            id_list=list(dict.fromkeys(id_list))
            id_not_in_DB = set(id_list)-set(self.database['uid'])
            if len(id_not_in_DB)>0:
                print('id not in DB:',len(id_not_in_DB),', you may want to updata your Database')
            id_list = list(set(id_list) & set(self.database['uid']))
            data={}
            data = pd.DataFrame(data)
            for uid in id_list:    
                # print(self.database.loc[self.database['uid'] == uid])
                data=pd.concat([data,self.database.loc[self.database['uid'] == uid]],axis=0)
            return data
        else:
            print('Run predict for all id in Database')
            return self.database

    def run_inference(self):
        
        ### load data------------------------------------------
        data = self.getdata()
        data.head()
        data.columns
        data.describe()
        data.dtypes
        data.columns.to_series().groupby(data.dtypes).groups
        data.info()
        data.isna().any()
        print(data["isChurn"].value_counts())

        ### Preprocess-------------------------------------------
        response = data["isChurn"]

        data=data.drop(['last_time_play',"isChurn"],axis=1)


        identity = data['uid']
        data=data.drop(['uid'],axis=1)
        # Step 14: Feature Scaling-----------------------------------------------------------------------

        sc_X = StandardScaler()
        data2 = pd.DataFrame(sc_X.fit_transform(data))
        data2.columns = data.columns.values
        data2.index = data.index.values
        # data = data2

        classifier = joblib.load('finalized_model.sav')
        pred = classifier.predict(data2)
        
        results = pd.concat([identity,data,response], axis = 1).dropna()

        results['predictions'] = pred 

        # final_results.loc[final_results['predictions'].isin('True')]
        # print(final_results['predictions'].value_counts())
        

    


In [20]:
getresult = inference()
getresult.run_inference()

How do you want to input:
Run predict for all id in Database
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5504 entries, 0 to 5503
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   uid                        5504 non-null   object 
 1   high_level                 5504 non-null   float64
 2   total_level                5504 non-null   float64
 3   time_to_play               5504 non-null   float64
 4   ball_popped                5504 non-null   float64
 5   ball_drop                  5504 non-null   float64
 6   shot                       5504 non-null   float64
 7   is_win                     5504 non-null   float64
 8   is_first_time_win          5504 non-null   float64
 9   powerball                  5504 non-null   float64
 10  number_of_sessions         5504 non-null   float64
 11  length_of_sessions         5504 non-null   float64
 12  interval_between_sessions  5504 non-null   